Mounting google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/',force_remount=True)


Importing required libraries

In [ ]:
import cv2
from skimage.transform import downscale_local_mean
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from tqdm.auto import tqdm
import shutil as sh


**Label PROCESSING**


In [ ]:
def convertTrainLabel():
    df = pd.read_csv('./gdrive/MyDrive/CV Hackathon/Training Data/Labels/Train_DefectBoxes_PrithviAI.csv')

    df['x_center'] = df['X']
    df['y_center'] = df['Y']
    df['classes'] = 0
    from tqdm.auto import tqdm
    import shutil as sh
    df = df[[df.columns[0],'X', 'Y', 'W', 'H','x_center','y_center','classes']]
    
    index = list(set(df['  image_id']))
    fault=[]
    source = 'train'
    if True:
        for fold in [0]:
            val_index = index[len(index)*fold//5:len(index)*(fold+1)//5]
            for name,mini in tqdm(df.groupby('  image_id')):
                if name in val_index:
                    path2save = 'val2017/'
                else:
                    path2save = 'train2017/'
                name2 = name.split('.')[0]
                if not os.path.exists('./gdrive/MyDrive/convertor/fold{}/labels/'.format(fold)+path2save):
                    os.makedirs('./gdrive/MyDrive/convertor/fold{}/labels/'.format(fold)+path2save)
                with open('./gdrive/MyDrive/convertor/fold{}/labels/'.format(fold)+path2save+name2+".txt", 'w+') as f:
                    row = mini[['classes','x_center','y_center','W','H']].astype(float).values
                    row = row.astype(str)
                    for j in range(len(row)):
                        text = ' '.join(row[j])
                        f.write(text)
                        f.write("\n")
                with open('./gdrive/MyDrive/convertor/fold{}/labels/'.format(fold)+path2save+name2+"r1.txt", 'w+') as f:
                    row = mini[['classes','x_center','y_center','W','H']].astype(float).values
                    for i in range(len(row)):
                        row[i][1] , row[i][2]= 1-row[i][1] , 1- row[i][2]
                    row = row.astype(str)
                    for j in range(len(row)):
                        text = ' '.join(row[j])
                        f.write(text)
                        f.write("\n")
                if not os.path.exists('./gdrive/MyDrive/convertor/fold{}/images/{}'.format(fold,path2save)):
                    os.makedirs('./gdrive/MyDrive/convertor/fold{}/images/{}'.format(fold,path2save))
                try:
                    sh.copy("./gdrive/MyDrive/CV Hackathon/Training Data/Images Unzipped/Images/{}".format(name),'./gdrive/MyDrive/convertor/fold{}/images/{}/{}'.format(fold,path2save,name))
                except:
                    
                    try :
                        sh.copy("./gdrive/MyDrive/CV Hackathon/Sample Images/Sample Images/bad_images/{}".format(name),'./gdrive/MyDrive/convertor/fold{}/images/{}/{}'.format(fold,path2save,name))
                    except:
                        fault.append(name)
    print(len(fault),fault)

**IMAGE AUGMENTATION**

In [ ]:
def rotate(source):
    #this function rotate image by 180 degree  by this our amount of data samples get multiplied by 2 
    path = source
    kl = os.listdir(path)
    for j in tqdm(kl):
        fin = os.path.join(path,j)
        img = cv2.imread(fin)
        rotated_image1 = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
        r1 = cv2.rotate(rotated_image1, cv2.ROTATE_90_CLOCKWISE)
        fin,_=os. path. splitext(fin)
        fin+='r1.jpg'
        cv2.imwrite(fin,r1,[int(cv2.IMWRITE_JPEG_QUALITY),100])


**IMAGE PREPROCESSING**

In [ ]:
def reshape_img(img):
    #reshaping images form(1000,4096,3) =>(1024,1024) standard for yolo
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)#converting image to black and white
    factor_1 = 4
    factor_2 = 1
    gray=downscale_local_mean(gray, factors=(factor_2, factor_1)).astype(int)# shape of image(1000,1024)
    #providing padding
    gray =  cv2.copyMakeBorder(gray,12,12,0,0,cv2.BORDER_CONSTANT ,value = 0 )#shape of image(1024,10224))
    return gray


def imgprcs(img):
    #Applying Edge detection
    
    edges = cv2.Canny(img,100,200)
    return edges


def sharp(img):
    #Applying Sharpening of image by high pass filter
    
    krnl = np.array([[-1,-1,-1],
                                [-1,9,-1],
                                [-1,-1,-1]])
    sharp_img= cv2.filter2D(img,-1,krnl)
    return sharp_img


def preprocessing(source,destination):
    path_s = source
    path_d = destination
    list_itm = os.listdir(source) # this will contain list of images
    for j in tqdm(list_itm):
        
        #converting them to jpg format for saving space 
          
        img_pth = os.path.join(path_s,j)
        fin,_=os.path.splitext(j)
        fin+='.jpg'
        fin = os.path.join(destination,fin)
        img = cv2.imread(img_pth)
        img = reshape_img(img)
        cv2.imwrite(fin,img,[int(cv2.IMWRITE_JPEG_QUALITY),100])
        img = cv2.imread(fin)
        img = sharp(img)
        img = imgprcs(img)       
        cv2.imwrite(fin,img,[int(cv2.IMWRITE_JPEG_QUALITY),100])


**Training By YoLo**

Importing yolo--version-5 library from github

In [ ]:
!git clone https://github.com/ultralytics/yolov5

Installing required libraries

In [ ]:
%cd yolov5
%pip install -qr ./requirements.txt

Training our Yolo model 10 epochs with inital pre trained wieghts of yolov5

In [ ]:
!python train.py --img 1024 --batch 6 --epochs 10 --data ../gdrive/MyDrive/convertor/fold1/data.yaml --weights yolov5l.pt --cache

Validating the best model form the above epochs with the max allowed overlap of 0.2


In [ ]:
!python val.py --img 1024 --data ../gdrive/MyDrive/convertor/fold0/data.yaml --weights ./runs/train/exp/weights/best.pt --iou 0.2 --half

running the provided dataset with the best fitted model keeping threshold as 0.2

In [ ]:
!python detect.py --img 1024 --source ../test001 --iou-thres 0.2 --save-txt --weights ./runs/train/exp22/weights/best.pt

**converting the output to CSV format for submission**

In [ ]:
image_id=[]
X=[]
Y=[]
H=[]
W=[]
for i in os.listdir('detect\exp2\labels'):
    filename = os.path.join('detect\exp2\labels',i)
    with open (filename ,'r') as rf:
        for line in rf:
            ints = line.split()
            i,_=os. path. splitext(i)
            i+='.png'
            image_id.append(i)
            X.append(ints[1])
            Y.append(ints[2])
            H.append(ints[3])
            W.append(ints[4])

dic ={}
dic['image_id'] = image_id
dic['X'] =X
dic['Y'] =Y
dic['H'] =H
dic['W'] =W

df = pd.DataFrame(dic)

df.to_csv('file1.csv')



In [ ]:
defectlist= list(df['image_id'])
allfile = os.listdir('test')
res=[]
for i in allfile:
    if i in defectlist:
        res.append(1)
    else:
        res.append(0)
    
    dic2 = {}
dic2['images_id']=allfile
dic2['defect_flag']=res

df2 = pd.DataFrame(dic2)
df2.to_csv('file2.csv')
